In [1]:
import hashlib
import copy

# Clase Nodo

Esta clase simula cada nodo del arbol, tiene un valor, un valor hasheado, un lado que define en que lado esta, un padre, un hermano, y un hijo derecho e izquierdo

In [2]:
class Node:
    def __init__(self, valor):
        
        self.valor = valor
        self.string = valor
        self.lado = None
        self.brother = None
        self.parent = None
        self.hash = None
        self.izquierda = None
        self.derecha = None

# MerkleTree

Esta clase simula el MerkleTree, recibe un funcion de hash y un string

## self.construir

Esta funcion lo que hace es recibir el string y a partir de este crear todas las hojas del arbol, separando el string.

Una vez que tenemos todas las hojas entramos a un while el cual lo que hace es ir concatenando cada par de hermanos hasheado hasta llegar al root, de esta manera obtenemos nuestro MerkleTree.

## self.get_root

Esta funcion retorna el nodo root del arbol, dado mi implementacion de construir es muy facil obtenerlo

## self.get_proof_for

Esta funcion recibe una hoja y retorn una prueba de que la hoja es parte del arbol, dado mi implementacion su funcionamiento es muy simple.

Primero encontramos el objeto de la hoja recibida, una vez encontrado retornamos su hermano, y pasamos a iterar sobre su nodo padre, retornamos su hermano y pasamos a iterar sobre su nodo padre... hasta llegar al nodo root.

In [3]:
class MerkleTree:
    def __init__(self, strings, hash_func):
        self.hash_func = hash_func
        self.leaves = []

        # contruimos el arbol desde el nodo root
        self.root = self.construir(strings)
        

    def get_root(self):
        # retornamos el nodo root
        return self.root.hash

    def get_proof_for(self, item):

        for leave in self.leaves:
            pruebas = []
            pruebas_texto = []
            # iteramos sobre cada hoja hasta encontrar el nodo que corresponde a ese item
            
            if self.hash_func(item) == leave.hash:
                nodo_actual = leave
                # iteramos hasta llegar al nodo root
                while nodo_actual.parent:
                    # apendeamos a las pruebas el hash de su hermano
                    pruebas.append((nodo_actual.brother.hash,nodo_actual.brother.lado))
                    pruebas_texto.append((nodo_actual.brother.string,nodo_actual.brother.lado))
                    nodo_actual = nodo_actual.parent
                return pruebas
        return None

    def construir(self,strings):
        print_table = []
        nodos = []

        # para cada string creamos un nodo hoja y creamos su valor de hash
        for s in strings:
            nuevo_nodo = Node(s)
            nuevo_nodo.hash = self.hash_func(s)
            nodos.append(nuevo_nodo)
            print_table.append(nuevo_nodo.hash)
            self.leaves.append(nuevo_nodo)

        # iteramos hasta que no nos queden nodos que concatenar
        while len(nodos) > 1:
            cambio = []
            print_table = []
            # concatenamos el nodo izquierdo y derecho
            for index in range(0, len(nodos), 2):
                izquierda = nodos[index]
                if index+1 < len(nodos):
                    derecha = nodos[index+1]
                else:
                    # si el nodo no tiene hermano creamos su hermano copiandose a si mismo
                    derecha = copy.copy(izquierda)
                derecha.lado = 'd'
                derecha.brother = izquierda
                izquierda.brother = derecha
                izquierda.lado = 'i'
                nuevo_hash = izquierda.hash + derecha.hash
                nuevo_nodo = Node(nuevo_hash)
                # concatenamos el nodo izquierdo y derecho
                nuevo_nodo.string = izquierda.string + derecha.string
                nuevo_nodo.hash = self.hash_func(nuevo_hash)
                nuevo_nodo.izquierda = izquierda
                nuevo_nodo.derecha = derecha
                nuevo_nodo.izquierda.parent = nuevo_nodo
                nuevo_nodo.derecha.parent = nuevo_nodo
                # apendeamos a una lista que simula el proximo nivel
                cambio.append(nuevo_nodo)
                print_table.append(nuevo_nodo.hash)
            # cambiamos a iterar en el nivel siguiente
            nodos = cambio
        return nodos[0]


# Verify

Esta funcion lo que hace es dado una prueba entregada y un item, verifica si este item es aprte del arbol del nodo root.

Su funcionamiento es el siguiente:

Para los primero dos items de la prueba, concatenamos sus hash, segun el orden de derecha e izquierda de los nodos, una vez concatenados, hasheamos su concatenacion, este valor retornado lo concatenamos el el siguiente item del array y lo hasheamos, siempre respetando el orden de derecha e izquierda de los nodos. Todo esto hasta que se termine el array de pruebas, una vez terminado, comparamos el resultado con el nodo root, y retornamos True o False en cada caso.

In [6]:
def verify(root , item , proof, hash_func):

    if len(proof) > 0:
    
        # concatenamos las dos primeras pruebas siguiendo el orden izquierda-derecha
        
        if proof[0][1] == 'i':
            actual = proof[0][0] + hash_func(item)
        else:
            actual = hash_func(item)  + proof[0][0]
            
        actual = hash_func(actual)

        
        # concatenamos el nodo actual con la prueba siguiente siguiendo el orden izquierda-derecha hasta que no queden mas pruebas
        for node in proof[1::]:
            if node[1] == 'i':
                actual = node[0] + actual
            else:
                actual = actual + node[0]
            
            actual = hash_func(actual)
    else:
        actual = hash_func(item)
    
    # comparamos el resultado con el nodo root y retornamos
    if actual == root:
        return True
    return False
            

